# পর্ব 2: ফেডারেটেড লার্নিং এর সাথে পরিচিতি

শেষ বিভাগে, আমরা পয়েন্টার টেনসরগুলো সম্পর্কে শিখেছি, যা গোপনীয়তা সংরক্ষণ করে ডিপ লার্নিং এর জন্য আমাদের প্রয়োজনীয় অন্তর্নিহিত অবকাঠামো তৈরি করে। এই বিভাগে আমরা দেখবো, কিভাবে প্রাথমিক সরঞ্জামসমূহ ব্যবহার করে আমাদের প্রথম গোপনীয়তা সংরক্ষণকারী ডিপ লার্নিং এল্গোরিদম - ফেডারেটেড লার্নিং তৈরি করতে পারি।

লেখক:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

অনুবাদক:
- Sayantan Das - Github: [@ucalyptus](https://github.com/ucalyptus)

- Mir Mohammad Jaber - Twitter: [@jabertuhin](https://twitter.com/jabertuhin)

### ফেডারেটড লার্নিং কী?

এটি ডিপ লার্নিং মডেলগুলি প্রশিক্ষণের একটি সহজ, শক্তিশালী উপায়। আপনি যদি প্রশিক্ষণের ডেটা সম্পর্কে চিন্তা করেন, এটি সর্বদা কোনও ধরণের সংগ্রহ প্রক্রিয়ার ফলাফল। লোকেরা (ডিভাইসের মাধ্যমে) বাস্তব বিশ্বে ঘটনা রেকর্ড করে ডেটা উৎপন্ন করে। সাধারণত, এই ডেটাটি একক, কেন্দ্রীয় অবস্থানে সংযুক্ত করা হয় যাতে আপনি একটি মেশিন লার্নিং মডেলকে প্রশিক্ষণ দিতে পারেন। ফেডারেটেড লার্নিং এটিকে নিজের মাথায় নিয়ে নেয়!

মডেলটিতে প্রশিক্ষণ ডেটা আনার পরিবর্তে (একটি কেন্দ্রীয় সার্ভার) আপনি মডেলটিকে প্রশিক্ষণের ডেটাতে নিয়ে আসুন (এটি যেখানেই থাকুক না কেন)।

ধারণাটি হলো এটি যার দ্বারা ডেটা তৈরি করছে সে কেবলমাত্র স্থায়ী অনুলিপিটির মালিক হতে পারে এবং যার ফলে এটিতে কখন কার অ্যাক্সেস রয়েছে তা নিয়ন্ত্রণ বজায় রাখে। খুব সুন্দর, তাই না?

# বিভাগ 2.1 - একটি খেলনা ফেডারেটেড লার্নিং-এর উদাহরণ

আসুন একটি খেলনা মডেলকে কেন্দ্রিক উপায়ে প্রশিক্ষণ দিয়ে শুরু করি। মডেলগুলি পাওয়ার সাথে সাথে এটি একটি সাধারণ। আমাদের প্রথম প্রয়োজন:

- একটি খেলনা ডেটাসেট
- একটি মডেল
- ডেটা ফিট করার জন্য একটি মডেল প্রশিক্ষণের কিছু প্রাথমিক প্রশিক্ষণের যুক্তি।

দ্রষ্টব্য: যদি এই API-টি আপনার কাছে অপরিচিত হয় - তবে [fast.ai](http://fast.ai) এ যান এবং এই টিউটোরিয়ালটি চালিয়ে যাওয়ার আগে তাদের কোর্সটি গ্রহণ করুন।

In [0]:
import torch
from torch import nn
from torch import optim

In [0]:

# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)

In [0]:
train()

এবং আপনার জন্য এখন প্রস্তুত! আমরা প্রচলিত পদ্ধতিতে একটি বেসিক মডেলকে প্রশিক্ষণ দিয়েছি। আমাদের সমস্ত ডেটা আমাদের স্থানীয় মেশিনে একত্রিত হয় এবং আমরা এটি আমাদের মডেলটিতে আপডেট করতে ব্যবহার করতে পারি। ফেডারেটেড লার্নিং অবশ্য এইভাবে কাজ করে না। সুতরাং, ফেডারেট লার্নিংয়ের উপায়ে এটি করার জন্য এই উদাহরণটি সংশোধন করি!

সুতরাং, আমাদের কী দরকার:

- কয়েকজন শ্রমিক তৈরি করুন
- প্রতিটি কর্মীর উপর প্রশিক্ষণ ডেটার পয়েন্টার পান
- ফেডারেট লার্নিংয়ের জন্য প্রশিক্ষণের যুক্তি আপডেট করা

     নতুন প্রশিক্ষণের ধাপসমূহ:
     - কর্মী সংশোধন করতে মডেল প্রেরণ করুন
     - সেখানে অবস্থিত ডেটার উপর প্রশিক্ষণ করুন
     - মডেলটি ফিরে পাবেন এবং পরবর্তী কর্মীর থেকে এর পুনরাবৃত্তি করুন

In [0]:
import syft as sy
hook = sy.TorchHook(torch)


In [0]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")


In [0]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [0]:
def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()

            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [0]:
train()

## সাবাশ!

এবং voilà! ফেডারেটেড লার্নিং ব্যবহার করে আমরা এখন একটি খুব সাধারণ ডিপ লার্নিং মডেল প্রশিক্ষণ দিচ্ছি! আমরা প্রতিটি কর্মীকে মডেল প্রেরণ করি, একটি নতুন গ্রেডিয়েন্ট উৎপন্ন করি এবং তারপরে গ্রেডিয়েন্টটি আমাদের স্থানীয় সার্ভারে ফিরিয়ে আনি যেখানে আমরা আমাদের গ্লোবাল মডেলটি আপডেট করি। এই প্রক্রিয়াতে কখনই আমরা অন্তর্নিহিত প্রশিক্ষণের ডেটা দেখতে বা দেখার জন্য অনুরোধ করি না! আমরা বব এবং অ্যালিসের গোপনীয়তা সংরক্ষণ করি !!!

## এই উদাহরণের ত্রুটিসমূহ

সুতরাং, যদিও এই উদাহরণটি ফেডারেটড লার্নিংয়ের সাথে পরিচিতিতে দুর্দান্ত, তবে এটির এখনও কিছু বড় ত্রুটি রয়েছে। এর মধ্যে উল্লেখযোগ্য হল, যখন আমরা `Model.get ()' কল করি এবং বব বা অ্যালিসের কাছ থেকে আপডেট হওয়া মডেলটি পাই, তখন আমরা বব এবং অ্যালিসের প্রশিক্ষণের ডেটা সম্পর্কে তাদের গ্রেডিয়েন্টগুলি দেখে আসলে অনেক কিছু জানতে পারি। কিছু ক্ষেত্রে, আমরা তাদের প্রশিক্ষণের ডেটা পুরোপুরি পুনরুদ্ধার করতে পারি!

তো, কী করার আছে? ঠিক আছে, প্রথম কৌশল যা লোকেরা প্রয়োগ করা থাকে তা হলো - **কেন্দ্রীয় সার্ভারে আপলোড করার আগে একাধিক ব্যক্তির মধ্যে গ্রেডিয়েন্ট গড় করা**। এই কৌশলটির জন্য পয়েন্টার টেন্সর অবজেক্টের আরও কিছু পরিশীলিত ব্যবহারের প্রয়োজন হবে। সুতরাং, পরবর্তী বিভাগে, আমরা আরও উন্নত পয়েন্টার কার্যকারিতা সম্পর্কে জানতে কিছুটা সময় নেব এবং তারপরে আমরা এই ফেডারেট লার্নিং উদাহরণটি আপগ্রেড করব।

# অভিনন্দন !!! - সম্প্রদায় যোগদানের সময়!

এই নোটবুক টিউটোরিয়ালটি সম্পন্ন করার জন্য অভিনন্দন! আপনি যদি এটি উপভোগ করেন এবং গোপনীয়তা সংরক্ষণ, এআই এবং এআই সরবরাহ চেইনের (ডেটা) বিকেন্দ্রীভূত মালিকানার দিকে আন্দোলনে যোগ দিতে চান, আপনি নিম্নলিখিত উপায়ে এটি করতে পারেন!

### গিটহাবে PySyft-কে স্টার দিন

আমাদের সম্প্রদায়কে সাহায্য করার সবচেয়ে সহজ উপায় হল রিপোজিটরিতে স্টার দেয়া! এটি আমরা যে দারুন সরঞ্জামগুলি তৈরি করছি তার সচেতনতা বাড়াতে সহায়তা করে।

- [Star PySyft](https://github.com/OpenMined/PySyft)

### আমাদের স্ল্যাকে যোগ দিন!

সর্বশেষতম অগ্রগতিতে নিজেকে আপ টু ডেট রাখার সর্বোত্তম উপায় হলো আমাদের সম্প্রদায়ে যোগদান করা! আপনি [http://slack.openmined.org](http://slack.openmined.org) এ ফর্মটি পূরণ করে এটি করতে পারেন

### একটি কোড প্রকল্পে যোগদান করুন!

আমাদের কমিউনিটিতে অবদান রাখার সেরা উপায় হলো একজন কোড অবদানকারীতে পরিণত হওয়া। যেকোন সময় আপনি PySyft এর গিটহাব ইস্যুর পেজে যেতে পারেন এবং "Projects" দিয়ে বাছাই করবেন। এর মাধ্যমে আপনি যে সকল প্রজেক্টে যোগদান করতে পারবেন সেগুলোর উপরের দিকের টিকেটের ওভারভিউ পাবেন। আপনি যদি কোন প্রজেক্টে জয়েন করতে ইচ্ছুক না হোন, কিন্তু কিছুটা কোডিং করতে ইচ্ছুক সেক্ষেত্রে আপনি "one off" মিনি প্রজেক্টগুলো দেখতে পারেন গিটহাব ইস্যুতে "good first issue" চিহ্নিত ইস্যুগুলো।

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)
### দান করুন

আপনার যদি আমাদের কোডবেজে অবদান রাখারা সময় না হয়, কিন্তু এরপরও আমাদেরকে সমর্থন দিতে চান তাহলে আমাদের উন্মুক্ত সংগ্রহের সমর্থক হতে পারেন। সকল ধরনের দানের অর্থ আমাদের ওয়েব হোস্টিং এবং অন্যান্য কমিউনিটি কার্যক্রমে খরচ হয় যেমন - হ্যাকাথন, মিটাপ।

[OpenMined's Open Collective Page](https://opencollective.com/openmined)